# Embeddings Quantization

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import uuid
import pandas as pd
from datasets import load_dataset
from fastembed import TextEmbedding
from qdrant_client import models, QdrantClient

from src import load_data, embeddings


c:\Users\manuelalberto.romero\Documents\repos\dslabs\mbit\peia-oct25\dslab-rag-retrieval\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Params

In [3]:
RETRIEVE_K=5
SMPL_SIZE = 25  # int or None

# Clients

In [4]:
client_qdrant = QdrantClient(":memory:") 


# Load and Prepare Data

In [5]:
df_ragquas = load_data.ragquas()
df_docs = load_data.prepare_ragquas(df_ragquas)
df_docs.info()

Raw shape: (250, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     183 non-null    int64 
 1   topic     183 non-null    object
 2   variant   183 non-null    object
 3   question  183 non-null    object
 4   answer    183 non-null    object
 5   j         183 non-null    int64 
 6   context_  183 non-null    object
 7   text_     183 non-null    object
 8   link_     183 non-null    object
 9   uuid      183 non-null    object
dtypes: int64(2), object(8)
memory usage: 14.4+ KB


In [6]:
if SMPL_SIZE:
    df_docs_smpl = df_docs.iloc[:SMPL_SIZE]
else:
    df_docs_smpl = df_docs

# Encode

In [7]:
encoder = TextEmbedding(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    )


dn_embeddings = list(encoder.embed(df_docs_smpl['text_']))

print(f"{len(dn_embeddings)=}")
print(f"{len(dn_embeddings[0])=}")

C:\Users\manuelalberto.romero\AppData\Local\Temp\ipykernel_46096\315642572.py:1: UserWarning: The model sentence-transformers/paraphrase-multilingual-mpnet-base-v2 now uses mean pooling instead of CLS embedding. In order to preserve the previous behaviour, consider either pinning fastembed version to 0.5.1 or using `add_custom_model` functionality.
  encoder = TextEmbedding(


len(dn_embeddings)=25
len(dn_embeddings[0])=768


In [8]:
lst_embeddings = []

for idx in range(len(dn_embeddings)):
    dn_vec = dn_embeddings[idx]

    lst_embeddings.append(
        dn_vec.tolist()
    )

lst_pts = embeddings.create_index_points(
    lst_embeddings,
    df_docs_smpl,
)

lst_qdrant_pts = embeddings.convert_to_qdrant_points(lst_pts)

print(f"{len(lst_qdrant_pts)=}")

len(lst_qdrant_pts)=25


# Index

In [9]:
client_qdrant.create_collection(
    collection_name='raw',
    vectors_config=models.VectorParams(
            size=encoder.embedding_size, 
            distance=models.Distance.COSINE,
        )
)

client_qdrant.create_collection(
    collection_name='q2',
    vectors_config=models.VectorParams(
            size=encoder.embedding_size, 
            distance=models.Distance.COSINE,
        ),
    quantization_config=models.BinaryQuantization(
        binary=models.BinaryQuantizationConfig(
            always_ram=True,
        ),
    ),
)

client_qdrant.upsert(
    collection_name='raw',
    points=lst_qdrant_pts
)

client_qdrant.upsert(
    collection_name='q2',
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

# Query

In [10]:
_q_text = df_docs_smpl.sample(1)['question']
_q_text

0    ¿Cuál es la forma más fácil de reclamar cuando...
Name: question, dtype: object

In [12]:
_q_vec = next(encoder.embed(_q_text))

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        "raw",
        _q_vec,
        retrieve_k=2,
)

Question: 0    ¿Cuál es la forma más fácil de reclamar cuando...
Name: question, dtype: object
point.id='ba8ef288-a2c5-4f79-9dc3-b5c812b546c4'
point.score=0.49639524378939925
Doc: 5 páginas donde poder reclamar tus vuelos por retraso o cancelación de manera gratuita
Irina Pérez

    En muchas ocasiones no se realiza la reclamación de un vuelo por retraso o cancelación, porque se desconocen los derechos como pasajeros y la manera de realizar el trámite. 
    Por ello, aquí van 5 reclamadores online que te ayudarán a conseguir una indemnización por tu vuelo, sin que tengas que desplazarte ni hacer papeleos.
    Descubre más historias en Business Insider España.

Volar es si...
idx: 0, j: 1
------------------------------
point.id='00417e00-bc59-4faf-ab14-3e73e197a8a9'
point.score=0.46102700078081527
Doc: Indemnización retraso vuelo

Navegación por el Retraso en el vuelo

    La indemnización por retraso de vuelo y los derechos en breve
    ¿Tu vuelo se ha retrasado? Condiciones de indemni

In [13]:
_q_vec = next(encoder.embed(_q_text))

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        "q2",
        _q_vec,
        retrieve_k=2,
)

Question: 0    ¿Cuál es la forma más fácil de reclamar cuando...
Name: question, dtype: object
point.id='ba8ef288-a2c5-4f79-9dc3-b5c812b546c4'
point.score=0.49639524378939925
Doc: 5 páginas donde poder reclamar tus vuelos por retraso o cancelación de manera gratuita
Irina Pérez

    En muchas ocasiones no se realiza la reclamación de un vuelo por retraso o cancelación, porque se desconocen los derechos como pasajeros y la manera de realizar el trámite. 
    Por ello, aquí van 5 reclamadores online que te ayudarán a conseguir una indemnización por tu vuelo, sin que tengas que desplazarte ni hacer papeleos.
    Descubre más historias en Business Insider España.

Volar es si...
idx: 0, j: 1
------------------------------
point.id='00417e00-bc59-4faf-ab14-3e73e197a8a9'
point.score=0.46102700078081527
Doc: Indemnización retraso vuelo

Navegación por el Retraso en el vuelo

    La indemnización por retraso de vuelo y los derechos en breve
    ¿Tu vuelo se ha retrasado? Condiciones de indemni